# Delete

We are going to see how to delete one or multiple rows of a table based on some filtering criteria

## Imports

In [11]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
from pathlib import Path
import tifffile as tiff

## Connection

In [12]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Loading the table from where we want to delete to check

In [13]:
table_name = 'measurements'

data = qrs.get_data(table_name)

data

,id_measurement,file_path_measurement,parent_measurement_id_measurement,measurementtype_id_measurement
0,98,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,4
1,99,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,98.0,4


## Filtering to decide what to delete

We want to delete all the measurements with the metadata key signal_type_measurement as amplitude.

In [8]:
filtered_data = data[data['id_measurement']>99]

filtered_data

,id_measurement,file_path_measurement,parent_measurement_id_measurement,measurementtype_id_measurement
2,100,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,NaN,2


## Deleting query

Now that we have only the rows that meet our deleting conditions, we can delete the measurements and their metadata with their ids

In [9]:
indexes = filtered_data['id_measurement'].values.tolist() #get the indexes to delete

# Convert indexes to a tuple for SQL query
if len(indexes) == 0:
    print("No indexes to delete.")
elif len(indexes) == 1:
    indexes = tuple((indexes[0],indexes[0]))  # Ensure tuple format for single index
else:
    indexes = tuple(indexes)


sql = f'delete from {table_name} where {table_name}.id in {indexes}'
print(sql)

# Add RESTART IDENTITY command
reset_sql = f'ALTER TABLE {table_name} RESTART IDENTITY'
print(f"Reset identity SQL: {reset_sql}")

delete from measurements where measurements.id in (100, 100)
Reset identity SQL: ALTER TABLE measurements RESTART IDENTITY


In [10]:
# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

## Check

If you execute the whole code again and the filtered table is empty, the delete whas succesfull.